# Assignment 2. Econophysics simulator

Economic inequality is one of the defining social issues of our age. Yet have a poor grasp of the scale of inequality,
as [described in Scientific American](https://www.scientificamerican.com/article/economic-inequality-it-s-far-worse-than-you-think/) 
and nicely shown in [this video](https://www.youtube.com/watch?v=QPKKQnijnsM):
[<img src="res/inequality.png" style="height:16em">](https://www.youtube.com/watch?v=QPKKQnijnsM)

How does inequality arise? Is it an inevitable outcome of liberal economics, and if so how can it be  mitigated by economic policy? These questions [have been studied by economists](https://link.springer.com/article/10.1140/epjst/e2016-60162-3) and more recently 
[by](https://phys.org/news/2007-04-world-economies-similarities-economic-inequality.html)
[physicists](https://arxiv.org/abs/1606.06051).
In this assignment you will investigate a simple "econophysics" model of inequality.

<div style="background-color:wheat">**Goal of the assignment.** This assignment tests your vectorized thinking. You will be asked to run simulations on a population of hundreds of thousands of individuals, over a few thousand timesteps. Your code _must_ use NumPy vectorized operations rather than iterating over the population. You may use Python iteration over timesteps.
<br><br>
The assignment will ask you to simulate a variety of scenarios. You can organize your code however you like: for example, you might like to write a general-purpose simulator that can be used to answer all parts of the assignment, by plugging in different functions for the different scenarios.
</div>

In [ ]:
# Set up the grader as described in Section 0.3, with section='assignment2'
!pip install ucamcl
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions/scicomp', section='assignment2')

## 1. Kinetic exchange model

Here is a simple model. There are $N$ individuals in the population, each with an initial wealth of &pound;1. Every timestep, we randomly group them into $N/2$ pairs. (Assume $N$ is even.) For every pair, we simulate an economic exchange, as follows. Let the two paired individuals have wealth $v$ and $w$, and update their wealth according to
$$
v_{\text{new}} = R(v+w),
\quad
w_{\text{new}} = (1-R)(v+w)
$$
where $R$ is a random number in $[0,1]$, chosen independently for every pair and at every timestep.
This model is loosely inspired by the physics of gases, in which two gas molecules exchange a random amount of energy whenever they collide.

We can measure inequality with the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient),
$$
G = 2\frac{\sum_i i w_{(i)}}{n \sum_i w_{i}} - \Big(1 + \frac{1}{n}\Bigr)
$$
where $w_{(1)}$ is the smallest value, $w_{(2)}$ the second smallest etc.

**Question 1.** The model needs us to randomly group the population into $N/2$ pairs. Write a function `pairs(N)` that returns a tuple `(m1,m2)` where `m1` and `m2` are both vectors of length $N/2$ consisting of integers in the range $[0,...,N-1]$. Interpret this as "`m1[i]` is paired with `m2[i]`". For example, if you run `pairs(6)`, you might get the output

> `(array[(3, 0, 1]), array([2, 4, 5]))`

The pairing should be random, every individual equally likely to be paired with every other individual. Test your function with the following code:

In [ ]:
q = GRADER.fetch_question('q1')
m1,m2 = pairs(q.n)
ans = {'n': np.unique(np.concatenate([m1,m2])), 's': np.std(np.abs(m1-m2))}
GRADER.submit_answer(q, ans)

**Question 2.** Write a function `kinetic_exchange(v,w)` which takes two wealth vectors `v` and `w`, and returns a tuple `(vnew, wnew)` with two new vectors, according to the kinetic exchange model. Test it with the following code:

In [ ]:
q = GRADER.fetch_question('q2')
v,w = np.linspace(1,5,q.n), np.linspace(1,2,q.n)**q.p
vnew,wnew = kinetic_exchange(v,w)
ans = {'m1': np.mean(vnew), 's2': np.std(wnew)}
GRADER.submit_answer(q, ans)

**Question 3.** Write a function `gini(w)` which takes a vector `w` and returns the Gini coefficient. Test it with the following code:

In [ ]:
q = GRADER.fetch_question('q3')
w = np.linspace(0,1,q.n)**q.p
g = gini(w)
GRADER.submit_answer(q, {'g': g})

**Question 4.** Write a function `sim(N, T)` which runs the kinetic exchange model on a population of $N$ individuals for $T$ timesteps. It should return a pair `(w, gs)` where `w` is the wealth vector after $T$ timesteps, and `gs` is a length $T$ vector where `gs[i]` is the Gini coefficient at timestep $i$. Test it with the following code:

In [ ]:
q = GRADER.fetch_question('q4')
w,gs = sim(q.n, q.t)
ans = {'gm': np.mean(gs[(q.t/2):]), 'gs': np.std(gs[(q.t/2):]), 'ws': np.std(w)}
GRADER.submit_answer(q, ans)

**Question 5.** Plot a graph with the Gini coefficient on the vertical axis and timestep on the horizontal axis.

## Extending the model

In the kinetic exchange model, the poorest and the richest might swap places after just one transaction, which isn't very likely. Consider a different model for exchange. As before, suppose that two individuals with wealth $v$ and $w$ respectively are paired, but now let their wealth be updated by
$$
v_{\text{new}} = v + R \min(v,w),
\quad
w_{\text{new}} = w - R \min(v,w)
$$
where $R$ is now a random number in $[-1,1]$, chosen independently for every pair at every timestep. The idea is that each party to the exchange puts up a certain amount of money, but no more than they can afford. Call this the _value transfer_ model.

We can extend this model to include government intervention. Suppose the government charges a tax of say 20% on every exchange, collects all the tax revenue every timestep, and distributes it evenly to the entire population. This redistribution ought to have the effect of reducing inequality. Call this the _taxed value transfer_ model. Here is a concrete example, for a population of size 6.
> 1. Initial wealth values are $[0,2,5,3,1,2]$
> 1. We pair adjacent individuals: $[0,2; 5,3; 1,2]$
> 1. Random exchange amounts pre-tax are $[0,0; 2.6,-2.6; -0.4,0.4]$
> 1. Exchange amounts post-tax are $[0,0; 2.08,-2.6; -0.4,0.32]$
> 1. Government revenue is $(2.6-2.08) + (0.4-0.32) = 0.6$
> 1. Government redistributes $0.6/6=0.1$ to each person
> 1. Change in wealth is $[0.1,0.1, 2.18,-2.5, -0.3,0.42]$.<br>
> 1. New wealth vector is $[0.1, 2.1, 7.18, 0.5, 0.7, 2.42]$.

Also, let's introduce another way to measure inequality. The Gini coefficient is unfamiliar to many people, and it's easier to communicate "The richest 1% of the population own $x$% of the wealth."

**Question 6.** Implement the taxed value transfer model. Test it as follows:

In [ ]:
q = GRADER.fetch_question('q6')
v,w = np.linspace(1,5,q.n), np.linspace(1,2,q.n)**q.p
# Run the taxed value transfer model with tax rate q.tr to get vnew,wnew
# Here, q.tr is in the range [0,1], so q.tr=0.2 means 20% tax rate.
ans = {'m1': np.mean(vnew), 's2': np.std(wnew)}
GRADER.submit_answer(q, ans)

**Question 7.** Implement the "top x%" measure. Test it as follows:

In [ ]:
q = GRADER.fetch_question('q7')
w = np.random.pareto(a=q.a, size=q.n) + q.loc
# For wealth vector w, find the proportion of wealth owned by the top q.x.
# Your answer z should be in the range [0,1], and q.x is in the range [0,1] too.
GRADER.submit_answer(q, {'z': z})

**Question 8.** Simulate the taxed value transfer model, and measure the Gini coefficient and the "richest 1%" statistic at each timestep. Plot these two measures, as a function of time, for tax rates 0%, 20% and 40%. Here is some plotting code you may like to use.

In [ ]:
# Run your simulation. Let t be the vector of timesteps
# Let g0,g20,g40 be the vectors of Gini coefficients.

# Plot a labelled chart for Gini coefficient, with three lines.
H,W = 2,1
plt.subplot(H, W, 1)
plt.plot(t, g0, label='no tax')
plt.plot(t, t20, label='tax 20%')
plt.plot(t, t40, label='tax 40%')
plt.ylabel('gini')
plt.xlabel('timestep')
plt.legend()
# Repeat for a second chart, showing "wealth owned by top 1%"
# ...
plt.show()

**Question 9.** Run your simulation long enough for the two measures to stabilize. What is the long-run Gini coefficient, and proportion of wealth owned by the top 1%? Test your answer with the following code.

In [ ]:
q = GRADER.fetch_question('q9')
# Simulate a population of size q.n, using a tax rate of q.tr.
GRADER.submit_answer(q, {'gini': gini_coeff, 'top1pc': prop_wealth})

## Measuring economic mobility

Some degree of inequality might be acceptable if economic mobility was high, i.e. if everyone had similar chances of reaching either end of the wealth distribution. Economic mobility is often measured by splitting the population into five equal brackets, and measuring the chance of moving between brackets. From the [Wikipedia article on economic mobility](https://en.wikipedia.org/wiki/Economic_mobility):
> However, in terms of relative mobility [a report](https://www.brookings.edu/research/economic-mobility-of-families-across-generations/) stated: "contrary to American beliefs about
> equality of opportunity, a child’s economic position is heavily influenced by that of his
> or her parents." 42% of children born to parents in the bottom fifth of the income
> distribution ("quintile") remain in the bottom, while 39% born to parents in the top fifth
> remain at the top. Only half of the generation studied exceeded their parents economic
> standing by moving up one or more quintiles.

Let's measure economic mobility by recording the wealth distribution at one timepoint, and again 20 timesteps later, splitting the two distributions into quintiles, and counting what fraction of the population moved by more than one quintile from beginning to end. (In each timestep a median individual might find their wealth increasing or decreasing by around 50%, so one timestep corresponds roughly to several years of human life.) For example, if we have a population of 5000 and we draw up a matrix $A$ where $A_{i j}$ is the number of people who stated in quintile $i$ and ended in quintile $j$, we might get
$$
A = \left( \begin{matrix}
344& 313& 243& 100&   0\\
266& 261& 302& 167&   4\\
212& 260& 225& 272&  31\\
147& 143& 183& 331& 196\\
 31&  23&  47& 130& 769
\end{matrix} \right)
$$
(A quick check: the row sums and column sums are all 1000.) The number who moved by more than one quintile is 1148, which is 23% of the population.

**Question 10.** In a perfectly mobile economy, where everyone has equal chance of reaching any quintile, what fraction of people move by more than one quintile?

In [ ]:
q = GRADER.fetch_question('q10')
GRADER.submit_answer(q, {'theory': your_answer})

**Question 11.** Implement a function that takes two vectors `v` and `w`, where `v[i]` and `w[i]` measure respectively the wealth of individual $i$ at the beginning and end of a time period, and computes the proportion of people who moved by more than one quintile. Hint: look up [`np.percentile`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html#numpy.percentile) and [`np.digitize`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.digitize.html). Test your code as follows:

In [ ]:
q = GRADER.fetch_question('q11')
v,w = np.arange(q.n)**q.a, np.arange(q.n)**q.a * np.random.random(q.n)
# Find the proportion of people who moved by more than one quintile
GRADER.submit_answer(q, {'socmob': your_answer})

**Question 12.** Simulate the taxed value transfer model, long enough for it to stabilize, and measure the wealth vector $w_0$. Run it $t$ timesteps further, and measure the wealth vector again, and compute the social mobility measure $m$. Plot a graph of $m$ as a function of $t$. Test your code as follows:

In [ ]:
q = GRADER.fetch_question('q12')
# Simulate population size q.n, taxrate q.tr. At time q.t later, measure social mobility score.
GRADER.submit_answer(q, {'socmob': your_answer})

# Optional extensions

**Question 13.** Inflation means that wealth grows at an exponential rate, which is likely to exacerbate inequality. Compare a taxed value transfer model with tax rate 20% and no inflation, to a model with inflation of 3% per timestep. How much would tax have to increase, to compensate for this inflation?

**Question 14.** The economist [Thomas Piketty argues](https://en.wikipedia.org/wiki/Capital_in_the_Twenty-First_Century) that we have entered an age where the return on capital is greater than the growth due to income, and that this leads to higher inequality. We could incorporate income into the model by assigning each individual $i$ a per-timestep income $g_i$, where the $g_i$ are randomly chosen _a priori_. If there were no exchange, then wealth would grow linearly, and so the Gini coefficient would remain equal to `gini(g)`. Investigate what happens when we combine income, inflation, and random exchange. How well correlated are income and wealth? Do you agree with Piketty?